In [1]:
import pandas as pd

In [56]:
reactions_df = pd.read_csv('data/reactions.csv')
reactions_df["Left"] = reactions_df["Left"].str.split(" // ")
reactions_df["Right"] = reactions_df["Right"].str.split(" // ")
reactions_df["In-Pathway"] = reactions_df["In-Pathway"].str.split(" // ")
reactions_df["Substrates"] = reactions_df["Substrates"].str.split(" // ")
reactions_df["Enzymatic-Reaction"] = reactions_df["Enzymatic-Reaction"].str.split(" // ")
reactions_df.set_index("Object ID", inplace=True)
reactions_df

,Reactions,Substrates,In-Pathway,Right,Reaction-Direction,Systematic-Name,Synonyms,Std-Reduction-Potential,Spontaneous?,Species,...,Cannot-Balance?,Common-Name,Credits,Data-Source,EC-Number,Enzymatic-Reaction,Enzymes-Not-Used,Equilibrium-Constant,Gibbs-0,Left
Object ID,,,,,,,,,,,,,,,,,,,,,
RXN-22670,RXN-22670,"[ACP, CPD-24905, R-3-Isocyanylbutanoyl-ACP, AM...",[RXN-22664],"[R-3-Isocyanylbutanoyl-ACP, AMP, PROTON]",PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,EC-6.3.2,NaN,NaN,NaN,NaN,"[CPD-24905, ACP]"
RXN-9464,RXN-9464,"[WATER, CPD-10139, CPD-8222]",NaN,[CPD-8222],PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,NaN,...,NaN,"(+)-(1S,4R)-limonene-1,2- epoxide hydrolase",NaN,NaN,EC-3.3.2.8,NaN,NaN,NaN,10.286999,"[CPD-10139, WATER]"
RXN-10899,RXN-10899,"[NAD, CPD-11868, CPD-11871, NADH, PROTON]",[PWY-6341],"[CPD-11871, NADH, PROTON]",LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,NaN,[ENZRXN-16944],NaN,NaN,-0.468170,"[CPD-11868, NAD]"
RXN-10901,RXN-10901,"[NAD, CPD-11870, CPD-11871, NADH, PROTON]",[PWY-6341],"[CPD-11871, NADH, PROTON]",LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,NaN,"[ENZRXN-16948, ENZRXN-16946]",NaN,NaN,-0.468170,"[CPD-11870, NAD]"
RXN-9861,RXN-9861,"[CPD-10639, 2-CL-MUCONATE, PROTON]",NaN,"[2-CL-MUCONATE, PROTON]",REVERSIBLE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,EC-5.5.1.7,"[ENZRXN-15675, ENZRXN-15624]",NaN,NaN,9.931763,[CPD-10639]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3.2.1.3-RXN,3.2.1.3-RXN,"[WATER, Glycogens, Glucopyranose]",NaN,"[Glycogens, Glucopyranose]",PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,NaN,...,T,NaN,SRI // caspi,NaN,EC-3.2.1.3,NaN,NaN,NaN,NaN,"[Glycogens, WATER]"
RXN-24124,RXN-24124,"[WATER, 13-and-16-Sial-Gal-N-glycans, Galactos...",NaN,"[Galactosylated-N-Glycan, N-ACETYLNEURAMINATE]",PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,O0-13 // amackie,NaN,EC-3.2.1.18,NaN,NaN,NaN,NaN,"[13-and-16-Sial-Gal-N-glycans, WATER]"
RXN-24125,RXN-24125,"[WATER, Galactosylated-N-Glycan, 24-26-N-linke...",NaN,"[24-26-N-linked-Glycan, GALACTOSE]",PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,O0-13 // amackie,NaN,NaN,NaN,NaN,NaN,NaN,"[Galactosylated-N-Glycan, WATER]"


In [62]:
reactions_df["Reaction-Direction"].value_counts()

Reaction-Direction
PHYSIOL-LEFT-TO-RIGHT    13061
LEFT-TO-RIGHT             4108
REVERSIBLE                1620
PHYSIOL-RIGHT-TO-LEFT      948
RIGHT-TO-LEFT              432
Name: count, dtype: int64

In [74]:
# if there is a reaction that transforms R to L (may need other substrates), we add an edge

edge_list = {}
for index, row in reactions_df.iterrows():
    
    if row["Reaction-Direction"] in ["LEFT-TO-RIGHT", "PHYSIOL-LEFT-TO-RIGHT", "REVERSIBLE"]:
        for l in row["Left"]:
            for r in row["Right"]:
                if (l, r) in edge_list:
                    edge_list[(l, r)].append(index)
                else:
                    edge_list[(l, r)] = [index]
    
    if row["Reaction-Direction"] in ["RIGHT-TO-LEFT", "PHYSIOL-RIGHT-TO-LEFT", "REVERSIBLE"]:
        for l in row["Right"]:
            for r in row["Left"]:
                if (l, r) in edge_list:
                    edge_list[(l, r)].append(index)
                else:
                    edge_list[(l, r)] = [index]

pd.Series(edge_list, name="Reactions").sort_values(ascending=False)

PROTON                    5-DEHYDROGLUCONATE      [YIAE1-RXN, TRANS-RXN0-228, TRANS-RXN0-228]
NADPH                     5-DEHYDROGLUCONATE                                      [YIAE1-RXN]
25-DIDEHYDRO-D-GLUCONATE  5-DEHYDROGLUCONATE                                      [YIAE1-RXN]
D-XYLULOSE                PROTON                                              [XYLULOKIN-RXN]
                          XYLULOSE-5-PHOSPHATE                                [XYLULOKIN-RXN]
                                                                     ...                     
NAD                       CPD-862                               [+-BORNEOL-DEHYDROGENASE-RXN]
CPD-1903                  PROTON                                [+-BORNEOL-DEHYDROGENASE-RXN]
                          NADH                                  [+-BORNEOL-DEHYDROGENASE-RXN]
                          CPD-862                               [+-BORNEOL-DEHYDROGENASE-RXN]
PROTON                    CPD-1903                          

In [60]:
substrates_count = {}

for substrates in reactions_df["Substrates"]:
    for substrate in substrates:
        if substrate in substrates_count:
            substrates_count[substrate] += 1
        else:
            substrates_count[substrate] = 1
            
pd.Series(substrates_count, name="Count").sort_values(ascending=False)

PROTON                    11069
WATER                      8065
OXYGEN-MOLECULE            3193
ATP                        1584
NADPH                      1481
                          ...  
CETRAXATE                     1
CETRAXATE-BENZYL-ESTER        1
CPD-9133                      1
CPD-16353                     1
CPD-20887                     1
Name: Count, Length: 17653, dtype: int64

In [38]:
reactions_df.set_index("Object ID").loc["RXN-22670"]

Reactions                                                            RXN-22670
Substrates                   ACP // CPD-24905 // R-3-Isocyanylbutanoyl-ACP ...
In-Pathway                                                           RXN-22664
Right                               R-3-Isocyanylbutanoyl-ACP // AMP // PROTON
Reaction-Direction                                       PHYSIOL-LEFT-TO-RIGHT
Systematic-Name                                                            NaN
Synonyms                                                                   NaN
Std-Reduction-Potential                                                    NaN
Spontaneous?                                                               NaN
Species                                                                    NaN
Signal                                                                     NaN
Requirements                                                               NaN
Regulated-By                                        

In [39]:
reactions_df.set_index("Object ID").loc["RXN-22664"]

Reactions                                                            RXN-22664
Substrates                   ATP // ACP // CPD-24900 // R-3-Isocyanylbutano...
In-Pathway                                                            PWY-8320
Right                                  R-3-Isocyanylbutanoyl-ACP // AMP // PPI
Reaction-Direction                                       PHYSIOL-LEFT-TO-RIGHT
Systematic-Name                                                            NaN
Synonyms                                                                   NaN
Std-Reduction-Potential                                                    NaN
Spontaneous?                                                               NaN
Species                                                                    NaN
Signal                                                                     NaN
Requirements                                                               NaN
Regulated-By                                        

In [ ]:
reactions_df.set_index("Object ID").loc["RXN-22664"]

In [75]:
pathways_df = pd.read_csv('data/pathways.csv')
pathways_df.set_index("Object ID")

,Pathways,Chimeric?,Common-Name,Credits,Data-Source,Engineered?,Enzymes-Not-Used,Hypothetical-Reactions,In-Pathway,Key-Reactions,...,Reaction-List,Regulated-By,Score,Species,Sub-Pathways,Synonyms,Taxonomic-Range,Variants?,Super-Pathways,Link
Object ID,,,,,,,,,,,,,,,,,,,,,
PWY-6990,PWY-6990,NaN,(+)-camphor biosynthesis,SRI // caspi,NaN,NaN,NaN,NaN,NaN,NaN,...,GERANYL-DIPHOSPHATE-CYCLASE-RXN // RXN-13046 /...,NaN,NaN,TAX-38868,NaN,(<i>R</i>)-camphor biosynthesis,TAX-33090,NaN,NaN,HTTPS://metacyc.org/pathway?orgid=META&id=PWY-...
PWY-2467,PWY-2467,NaN,(+)-pisatin biosynthesis,SRI // caspi // O-28 // THE-ARABIDOPSIS-INFORM...,NaN,NaN,NaN,RXN-18746 // RXN-18745 // RXN-16377 // RXN-2054,NaN,NaN,...,RXN-18746 // RXN-18745 // RXN-16377 // RXN-163...,NaN,NaN,TAX-3888,NaN,NaN,TAX-3803,NaN,NaN,HTTPS://metacyc.org/pathway?orgid=META&id=PWY-...
PWY-7487,PWY-7487,NaN,(+)-secoisolariciresinol diglucoside biosynthesis,O-28 // hartmut,NaN,NaN,NaN,NaN,NaN,NaN,...,RXN-15443 // RXN-15442,NaN,NaN,TAX-4006,NaN,NaN,TAX-3398,NaN,NaN,HTTPS://metacyc.org/pathway?orgid=META&id=PWY-...
PWY-7749,PWY-7749,NaN,(-)-4'-demethyl-epipodophyllotoxin biosynthesis,SRI // caspi,NaN,NaN,NaN,NaN,NaN,NaN,...,RXN-17350 // RXN-17349 // RXN-16344,NaN,NaN,TAX-93608,NaN,etoposide aglycone biosynthesis // (-)-4'-desm...,TAX-3398,NaN,NaN,HTTPS://metacyc.org/pathway?orgid=META&id=PWY-...
PWY-6991,PWY-6991,NaN,(-)-camphor biosynthesis,SRI // caspi,NaN,NaN,NaN,NaN,NaN,NaN,...,RXN-13002 // RXN-13047 // --BORNEOL-DEHYDROGEN...,NaN,NaN,TAX-301877 // TAX-128002,NaN,(<i>S</i>)-camphor biosynthesis,TAX-33090,NaN,NaN,HTTPS://metacyc.org/pathway?orgid=META&id=PWY-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PWY-6288,PWY-6288,NaN,zeaxanthin-&beta;-D-diglucoside biosynthesis,SRI // caspi,NaN,NaN,NaN,NaN,NaN,NaN,...,RXN-10673 // RXN-10672,NaN,NaN,TAX-549 // TAX-553,NaN,NaN,TAX-2,NaN,NaN,HTTPS://metacyc.org/pathway?orgid=META&id=PWY-...
PWY-6265,PWY-6265,NaN,zerumbone biosynthesis,O-28 // pujar,NaN,NaN,NaN,NaN,NaN,NaN,...,RXN-10629 // RXN-10628 // RXN-8415,NaN,NaN,TAX-311405,NaN,NaN,TAX-4642,NaN,NaN,HTTPS://metacyc.org/pathway?orgid=META&id=PWY-...
PWY-7694,PWY-7694,NaN,zwittermicin A biosynthesis,SRI // caspi,NaN,NaN,NaN,NaN,NaN,NaN,...,RXN-16916 // RXN-16915 // RXN-18393 // RXN-183...,NaN,NaN,TAX-1396 // TAX-1428,NaN,NaN,TAX-2,NaN,NaN,HTTPS://metacyc.org/pathway?orgid=META&id=PWY-...


In [43]:
reactions_df.loc[reactions_df["In-Pathway"] == "PWY-8322"]

,Object ID,Reactions,Substrates,In-Pathway,Right,Reaction-Direction,Systematic-Name,Synonyms,Std-Reduction-Potential,Spontaneous?,...,Cannot-Balance?,Common-Name,Credits,Data-Source,EC-Number,Enzymatic-Reaction,Enzymes-Not-Used,Equilibrium-Constant,Gibbs-0,Left
13602,RXN-22686,RXN-22686,Man-12-Man-16-Man8GlcNAc2-proteins // GDP-MANN...,PWY-8322,Man-13-Man-12-Man-16-Man8GlcNAc2-proteins // G...,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,EC-2.4.1,ENZRXN-30419,NaN,NaN,NaN,GDP-MANNOSE // Man-12-Man-16-Man8GlcNAc2-proteins
13604,RXN-22685,RXN-22685,Man-16-Man8GlcNAc2-proteins // GDP-MANNOSE // ...,PWY-8322,Man-12-Man-16-Man8GlcNAc2-proteins // GDP // P...,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,EC-2.4.1,NaN,NaN,NaN,NaN,GDP-MANNOSE // Man-16-Man8GlcNAc2-proteins


In [48]:
reactions_df.loc[reactions_df["In-Pathway"].apply(lambda x: "PWY-8320" in str(x))] 

,Object ID,Reactions,Substrates,In-Pathway,Right,Reaction-Direction,Systematic-Name,Synonyms,Std-Reduction-Potential,Spontaneous?,...,Cannot-Balance?,Common-Name,Credits,Data-Source,EC-Number,Enzymatic-Reaction,Enzymes-Not-Used,Equilibrium-Constant,Gibbs-0,Left
1253,RXN-22664,RXN-22664,ATP // ACP // CPD-24900 // R-3-Isocyanylbutano...,PWY-8320,R-3-Isocyanylbutanoyl-ACP // AMP // PPI,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,EC-6.2.1.47,ENZRXN-30405,NaN,NaN,NaN,CPD-24900 // ACP // ATP
2134,RXN-22658,RXN-22658,OXYGEN-MOLECULE // 2-KETOGLUTARATE // CPD-2489...,PWY-8320 // RXN-22657,CPD-24901 // SUC // CARBON-DIOXIDE,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,NaN,NaN,NaN,NaN,NaN,CPD-24899 // 2-KETOGLUTARATE // OXYGEN-MOLECULE
2137,RXN-22659,RXN-22659,PROTON // OXYGEN-MOLECULE // 2-KETOGLUTARATE /...,PWY-8320 // RXN-22657,CPD-24900 // SUC // CARBON-DIOXIDE // WATER,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,NaN,NaN,NaN,NaN,NaN,CPD-24901 // 2-KETOGLUTARATE // OXYGEN-MOLECUL...
11751,RXN-22660,RXN-22660,ATP // ACP // CROTONATE // Crotonyl-ACP // AMP...,PWY-8320,Crotonyl-ACP // AMP // PPI,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,EC-6.2.1.47,ENZRXN-30402,NaN,NaN,NaN,CROTONATE // ACP // ATP
11756,RXN-22663,RXN-22663,WATER // GLY // Crotonyl-ACP // CPD-24899 // A...,PWY-8320,CPD-24899 // ACP // PROTON,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,EC-4.3.2.11,ENZRXN-30403,NaN,NaN,NaN,Crotonyl-ACP // GLY // WATER
15206,RXN-22666,RXN-22666,ATP // holo-L-Lysyl-Carrier-Proteins // LYS //...,PWY-8320,L-Lysyl-Lysyl-Carrier-Proteins // AMP // PPI,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,EC-6.2.1.an,ENZRXN-30407,NaN,NaN,NaN,LYS // holo-L-Lysyl-Carrier-Proteins // ATP
15215,RXN-22665,RXN-22665,NADPH // R-3-Isocyanylbutanoyl-ACP // L-Lysyl-...,PWY-8320,CPD-24902 // holo-L-Lysyl-Carrier-Proteins // ...,PHYSIOL-LEFT-TO-RIGHT,NaN,NaN,NaN,NaN,...,NaN,NaN,SRI // caspi,NaN,EC-2.3.1.dz,ENZRXN-30406,NaN,NaN,NaN,L-Lysyl-Lysyl-Carrier-Proteins // R-3-Isocyany...
